**FUNCTIONS USED IN NUMPY:**

```python
np.array(), np.empty(), np.arange(), np.linspace(), np.ones(), np.zeros(), np.pad(), np.sum(), np.absolute(), np.reshape(), np.round(), np.fft.fft(), np.roll(), np.atleast_2d(), np.where()
```

In [2]:
import numpy as np
import matplotlib.pyplot as plt

**N-POINT DFT USING TWIDDLE FACTOR MATRIX**

In [3]:
N = 4
D = np.empty(shape = (N,N),dtype = np.cdouble)
n = np.arange(N)
W = np.exp(-1j*2*np.pi/N)
for k in np.arange(N):
    D[k,:] = W**(k*n)
np.round(D)
x_n = np.atleast_2d(np.array([1,2,3,4])).transpose()
dft_n = np.zeros(shape=(N,1))
dft_n = np.round(D@x_n,3)
print(dft_n)
print(np.fft.fft(x_n.flatten())) #Input Goes in Like A List

[[10.+0.j]
 [-2.+2.j]
 [-2.-0.j]
 [-2.-2.j]]
[10.+0.j -2.+2.j -2.+0.j -2.-2.j]


**PARSEVALS RELATION**

In [4]:
print(np.sum(np.absolute(dft_n)**2)/N)
print(np.sum(np.absolute(x_n)**2))

30.0
30


**CIRCULAR CONVOLUTION**

In [5]:
N = 4
x_n = np.array([1,2,3,4])
h_n = np.atleast_2d(np.array([2,1,2,1])).T
x_matrix = np.empty(shape = (N,N), dtype=np.cdouble)
for i in np.arange(N):
    x_matrix[:,i] = np.roll(x_n,i)
y_n = x_matrix@h_n
print(y_n)

[[14.+0.j]
 [16.+0.j]
 [14.+0.j]
 [16.+0.j]]


**LINEAR CONVOLUTION**

In [6]:
x1_n = np.array([1,2,3])
x2_n = np.array([4,5,6])

length1, length2 = len(x1_n), len(x2_n)
if length1 > length2:
    zero_padding = length1 - 1
    x2_n = np.flip(x2_n)
    x2_n = np.pad(x2_n,(0,zero_padding),'constant', constant_values=(0,0))
else:
    zero_padding = length2 - 1
    x1_n = np.flip(x1_n)
    x1_n = np.pad(x1_n,(0,zero_padding),'constant', constant_values=(0,0))
    x1_n, x2_n = x2_n, x1_n
y_n = np.zeros(length1 + length2 - 1)

for i in range(len(y_n)):
    y_n[i] = np.sum(x1_n * x2_n[length2-1:])
    x2_n = np.roll(x2_n,1)
print(np.round(y_n))

[ 4. 13. 28. 27. 18.]


**FAST FOURIER TRANSFORM USING DIT**

In [7]:
def FFT(g):
    N = len(g)
    if N<=1:
        return g
    even = FFT(g[0::2])
    odd = FFT(g[1::2])
    temp = np.zeros(N).astype(np.complex64)
    for n in range(N//2):
        temp[n] = even[n] + np.exp(-2j*np.pi*n/N)*odd[n]
        temp[n+N//2] = even[n] - np.exp(-2j*np.pi*n/N)*odd[n]
    return temp
print(FFT([1,2,3,4]))

[10.+0.j -2.+2.j -2.+0.j -2.-2.j]


**INVERSE FAST FOURIER TRANSFORM**

In [8]:
def IFFT(g):
    N = len(g)
    if N<=1:
        return g
    even = IFFT(g[0::2])
    odd = IFFT(g[1::2])
    temp = np.zeros(N).astype(np.complex64)
    for n in range(N//2):
        temp[n] = even[n] + np.exp(2j*np.pi*n/N)*odd[n]
        temp[n+N//2] = even[n] - np.exp(2j*np.pi*n/N)*odd[n]
    return temp
print(IFFT([10.+0.j,-2.+2.j,-2.+0.j,-2.-2.j])/N)

[1.+0.000000e+00j 2.+6.123234e-17j 3.+0.000000e+00j 4.-6.123234e-17j]


**OVERLAP AND SAVE**

In [10]:
def next_power_of_2(n):
    return 1 << (int(np.log2(n - 1)) + 1)

def pad_zeros_to(x, new_length):
    output = np.zeros((new_length,))
    output[:x.shape[0]] = x
    return output

def overlap_save_convolution(x, h, B):
    M = len(x)
    N = len(h)
    if K is None:
        K = max(B, next_power_of_2(N))
    num_input_blocks = np.ceil(M / B).astype(int) \
                     + np.ceil(K / B).astype(int) - 1
    xp = pad_zeros_to(x, num_input_blocks*B)
    output_size = num_input_blocks * B + N - 1
    y = np.zeros((output_size,))
    xw = np.zeros((K,))
    for n in range(num_input_blocks):
        xb = xp[n*B:n*B+B]
        xw = np.roll(xw, -B)
        xw[-B:] = xb
        u = np.convolve(xw,h)
        y[n*B:n*B+B] = u[-B:]
    return y[:M+N-1]

**OVERLAP AND ADD**

In [ ]:
def pad_zeros_to(x, new_length):
    output = np.zeros((new_length,))
    output[:x.shape[0]] = x
    return output

def overlap_add_convolution(x, h, B):
    M = len(x)
    N = len(h)
    num_input_blocks = np.ceil(M / B).astype(int)
    xp = pad_zeros_to(x, num_input_blocks*B)
    output_size = num_input_blocks * B + N - 1
    y = np.zeros((output_size,))
    for n in range(num_input_blocks):
        xb = xp[n*B:(n+1)*B]
        u = np.convolve(xb,h)
        y[n*B:n*B+len(u)] += u
    return y[:M+N-1]